对预处理的中英文文本进行统计分析，包括计算在收集样本上英语字母和单词或汉字的概率和熵，并利用收集的英文文本验证齐夫定律(Zipf’s law)，分析过程中设置多个比例的文本量进行分析，其中针对10%、20%、50%和100%四个档位的文本量绘制相应的图片具体展示。

In [1]:
# 库导入
import numpy as np
import pandas as pd
import collections as ct
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
'''分别为中文和英文设计不同的文本处理函数，对于中文直接对汉字进行相关的统计分析，而对于英文则分别进行字母和单词的统计分析，此外还有用于计算熵的函数。'''

# 中文文本处理函数
def zh_process(text):
    character_counts = ct.Counter(text)
    character_stats = pd.DataFrame(
        character_counts.items(), 
        columns=['character', 'count']
    )   # 构造一个DataFrame，包含汉字及其计数，df形式方便后续分析与作图
    total_characters = character_stats['count'].sum()   # 统计总汉字数
    character_stats['prob'] = character_stats['count'] / total_characters   # 计算每个汉字的概率
    return character_stats, total_characters    # 返回包含汉字、计数、概率和熵的DataFrame以及语料总汉字数

# 英文文本处理函数
def en_process(text):
    letter_counts = ct.Counter(c for c in text if c.isalpha())  # 统计字母
    word_counts = ct.Counter(text.split())  # 统计单词
    letter_stats = pd.DataFrame(
        letter_counts.items(),
        columns=['letter', 'count']
    )
    word_stats = pd.DataFrame(
        word_counts.items(),
        columns=['word', 'count']
    )   # 分别构造字母和单词的DataFrame
    total_letters = letter_stats['count'].sum()
    total_words = word_stats['count'].sum()
    letter_stats['prob'] = letter_stats['count'] / total_letters
    word_stats['prob'] = word_stats['count'] / total_words
    return letter_stats, word_stats, total_letters, total_words # 返回字母和单词的DataFrame及其总数

# 熵函数  
def calculate_entropy(dataframe):
    entropy = sum(-p * np.log2(p) for p in dataframe['prob'] if p > 0)
    return entropy

中文语料分析

In [3]:
zh_file_path = 'zh_wikipedia.txt'   # 设置文件路径
with open(zh_file_path, 'r', encoding='utf-8') as f:
    zh_text = f.read()

# 把语料分为按照10%、20%、30%、40%...100%进行分析
zh_segments = [zh_text[:int(len(zh_text) * i / 10)] for i in range(1, 11)]

# 构建多个DataFrame，用来存储不同文本量下的统计结果
zh_stats_list = []
zh_count_list = []
zh_entropy_list = []

# 利用循环，挨个进行分析，并储存下来处理的结果
for i, segment in enumerate(zh_segments, 1):
    print(10 * i, '%')
    zh_stats, zh_count = zh_process(segment)
    print(zh_count)
    zh_entropy = calculate_entropy(zh_stats)
    print(zh_entropy)

    # 将当前结果添加到列表中
    zh_stats_list.append(zh_stats)
    zh_count_list.append(zh_count)
    zh_entropy_list.append(zh_entropy)
    
zh_stats_total = zh_stats_list[-1]  # 获取完整文本的统计结果用于后续作图

10 %
828329
9.79952493649018
20 %
1656658
9.803411002287863
30 %
2484987
9.809508135707087
40 %
3313316
9.816796186353296
50 %
4141645
9.822400841935904
60 %
4969974
9.823540882222657
70 %
5798303
9.829783104156189
80 %
6626632
9.8283464107613
90 %
7454961
9.826826556570781
100 %
8283290
9.831362556659824


In [4]:
# 绘制汉字熵随文本量变化的图像

plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.plot(range(1, 11), zh_entropy_list, marker='o')
plt.title('汉字熵随文本量变化的图像', fontsize=16)
plt.xlabel('文本量（%）', fontsize=14)
plt.ylabel('汉字熵', fontsize=14)

# 显示数值
for i, entropy in enumerate(zh_entropy_list, 1):
    plt.text(i, entropy, f'{entropy:.4f}', fontsize=10, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('zh_entropy_vs_text_size.png', dpi=300, bbox_inches='tight')
plt.close()

In [5]:
# 按照出现次数排序汉字
zh_stats = zh_stats.sort_values(by='count', ascending=False).reset_index(drop=True)
zh_stats.head(20)

# 绘制汉字概率分布图（前20个汉字）
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='character', y='prob', data=zh_stats.head(20))
plt.title('全文本汉字出现概率分布图（前20个汉字）', fontsize=16)
plt.xlabel('汉字', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(zh_stats['prob'].head(20), 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

#plt.show()
plt.tight_layout()
plt.savefig('zh_top20_char_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [6]:
# 按照出现次数排序汉字
zh_stats_list[0] = zh_stats_list[0].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制汉字概率分布图（前20个汉字）
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='character', y='prob', data=zh_stats_list[0].head(20))
plt.title('10%文本汉字出现概率分布图（前20个汉字）', fontsize=16)
plt.xlabel('汉字', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(zh_stats_list[0]['prob'].head(20), 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

#plt.show()
plt.tight_layout()
plt.savefig('zh_top20_char_distribution10%.png', dpi=300, bbox_inches='tight')
plt.close()

In [7]:
# 按照出现次数排序汉字
zh_stats_list[1] = zh_stats_list[1].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制汉字概率分布图（前20个汉字）
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='character', y='prob', data=zh_stats_list[1].head(20))
plt.title('20%全文本汉字出现概率分布图（前20个汉字）', fontsize=16)
plt.xlabel('汉字', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(zh_stats_list[1]['prob'].head(20), 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

#plt.show()
plt.tight_layout()
plt.savefig('zh_top20_char_distribution20%.png', dpi=300, bbox_inches='tight')
plt.close()

In [8]:
# 按照出现次数排序汉字
zh_stats_list[4] = zh_stats_list[4].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制汉字概率分布图（前20个汉字）
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='character', y='prob', data=zh_stats_list[4].head(20))
plt.title('20%全文本汉字出现概率分布图（前20个汉字）', fontsize=16)
plt.xlabel('汉字', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(zh_stats_list[4]['prob'].head(20), 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

#plt.show()
plt.tight_layout()
plt.savefig('zh_top20_char_distribution50%.png', dpi=300, bbox_inches='tight')
plt.close()

In [9]:
zh_stats_list[9] = zh_stats_list[9].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制汉字的10%、20%、50%和100%文本量时候的四张前20汉字概率分布图，用矩阵图展示
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

fig, axs = plt.subplots(2, 2, figsize=(20, 13))
# 设置子图标题
titles = ['10%文本量', '20%文本量', '50%文本量', '全文本量']
data_indices = [0, 1, 4, 9]
for ax, title, idx in zip(axs.flatten(), titles, data_indices):
    sns.barplot(x='character', y='prob', data=zh_stats_list[idx].head(20), ax=ax)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('汉字', fontsize=14)
    ax.set_ylabel('概率', fontsize=14)
    # 显示数值
    for i, prob in enumerate(zh_stats_list[idx]['prob'].head(20), 1):
        ax.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('zh_top20_char_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

英文语料分析

In [10]:
en_file_path = 'en_wikipedia.txt'
with open(en_file_path, 'r', encoding='utf-8') as f:
    en_text_total = f.read()

words = en_text_total.split()
total_words = len(words)

# 把语料分为按照10%、20%、30%、40%...100%的词量进行分析
en_segments = [words[:int(len(words) * i / 10)] for i in range(1, 11)]

# 构建多个DataFrame，用来存储不同文本量下的统计结果
en_letter_stats_list = []
en_word_stats_list = []
en_letter_count_list = []
en_word_count_list = []
en_letter_entropy_list = []
en_word_entropy_list = []

# 利用循环，挨个进行分析，并储存下来处理的结果
for i, segment in enumerate(en_segments, 1):
    print(10 * i, '%')
    segment_text = ' '.join(segment)
    en_letter_stats, en_word_stats, en_letter_count, en_word_count = en_process(segment_text)
    print(en_word_count)
    en_word_entropy = calculate_entropy(en_word_stats)
    print(en_word_entropy)
    en_letter_entropy = calculate_entropy(en_letter_stats)
    print(en_letter_entropy)

    # 将当前结果添加到列表中
    en_letter_stats_list.append(en_letter_stats)
    en_word_stats_list.append(en_word_stats)
    en_letter_count_list.append(en_letter_count)
    en_word_count_list.append(en_word_count)
    en_word_entropy_list.append(en_word_entropy)
    en_letter_entropy_list.append(en_letter_entropy)

en_word_stats_total = en_word_stats_list[-1]  # 获取完整文本的统计结果用于后续作图
en_letter_stats_total = en_letter_stats_list[-1]

10 %
843055
10.83679149206672
4.179617029079061
20 %
1686110
10.916984726078098
4.179684288289823
30 %
2529166
10.95016877983413
4.179268782280771
40 %
3372221
10.98640578321347
4.1800134784637475
50 %
4215277
11.000954628015053
4.179839196212444
60 %
5058332
11.015494108718181
4.179634874630656
70 %
5901387
11.03180057591904
4.179696978791808
80 %
6744443
11.043846033057106
4.179683136422841
90 %
7587498
11.055347062882431
4.179818009048096
100 %
8430554
11.064286075640508
4.179788158071513


In [11]:
# 绘制英文字母熵随文本量变化的图像

plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.plot(np.arange(1, 11) * 10, en_letter_entropy_list, marker='o')
plt.title('英文字母熵随文本量变化的图像', fontsize=16)
plt.xlabel('文本量（%）', fontsize=14)
plt.ylabel('英文字母熵', fontsize=14)

# 显示数值
for i, entropy in enumerate(en_letter_entropy_list, 1):
    plt.text(i * 10, entropy, f'{entropy:.4f}', fontsize=10, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('en_letter_entropy_vs_text_size.png', dpi=300, bbox_inches='tight')
plt.close()

In [12]:
# 绘制英文单词熵随文本量变化的图像

plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.plot(np.arange(1, 11) * 10, en_word_entropy_list, marker='o')
plt.title('英文单词熵随文本量变化的图像', fontsize=16)
plt.xlabel('文本量（%）', fontsize=14)
plt.ylabel('英文单词熵', fontsize=14)

# 显示数值
for i, entropy in enumerate(en_word_entropy_list, 1):
    plt.text(i * 10, entropy, f'{entropy:.4f}', fontsize=10, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('en_word_entropy_vs_text_size.png', dpi=300, bbox_inches='tight')
plt.close()

In [13]:
# 按照出现次数排序字母
en_letter_stats_total = en_letter_stats_total.sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘图
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(10, 6))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='letter', y='prob', data=en_letter_stats_total)
plt.title('英文字母出现概率分布图', fontsize=16)
plt.xlabel('字母', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(en_letter_stats_total['prob'], 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('en_letter_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [14]:
# 按照出现次数排序单词
en_word_stats_total = en_word_stats_total.sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘图
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(10, 6))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.barplot(x='word', y='prob', data=en_word_stats_total.head(20))
plt.title('英文单词出现概率分布图', fontsize=16)
plt.xlabel('单词', fontsize=14)
plt.ylabel('概率', fontsize=14)

# 显示数值
for i, prob in enumerate(en_word_stats_total.head(20)['prob'], 1):
    plt.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('en_word_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

In [15]:
en_letter_stats_list[0] = en_letter_stats_list[0].sort_values(by='count', ascending=False).reset_index(drop=True)
en_letter_stats_list[1] = en_letter_stats_list[1].sort_values(by='count', ascending=False).reset_index(drop=True)
en_letter_stats_list[4] = en_letter_stats_list[4].sort_values(by='count', ascending=False).reset_index(drop=True)
en_letter_stats_list[9] = en_letter_stats_list[9].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制英文字母的10%、20%、50%和100%文本量时候的四张字母概率分布图，用矩阵图展示
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

fig, axs = plt.subplots(2, 2, figsize=(22, 13))
# 设置子图标题
titles = ['10%文本量', '20%文本量', '50%文本量', '全文本量']
data_indices = [0, 1, 4, 9]
for ax, title, idx in zip(axs.flatten(), titles, data_indices):
    sns.barplot(x='letter', y='prob', data=en_letter_stats_list[idx], ax=ax)
    ax.set_title(title, fontsize=18)
    ax.set_xlabel('字母', fontsize=14)
    ax.set_ylabel('概率', fontsize=14)
    # 把坐标轴数值或者字母写大一点
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    
    # 显示数值
    for i, prob in enumerate(en_letter_stats_list[idx]['prob'], 1):
        ax.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')
        
plt.tight_layout()
plt.savefig('en_letter_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

In [29]:
en_word_stats_list[0] = en_word_stats_list[0].sort_values(by='count', ascending=False).reset_index(drop=True)
en_word_stats_list[1] = en_word_stats_list[1].sort_values(by='count', ascending=False).reset_index(drop=True)
en_word_stats_list[4] = en_word_stats_list[4].sort_values(by='count', ascending=False).reset_index(drop=True)
en_word_stats_list[9] = en_word_stats_list[9].sort_values(by='count', ascending=False).reset_index(drop=True)

# 绘制英文单词的10%、20%、50%和100%文本量时候的四张排名前20单词概率分布图，用矩阵图展示
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

fig, axs = plt.subplots(2, 2, figsize=(22, 13))
# 设置子图标题
titles = ['10%文本量', '20%文本量', '50%文本量', '全文本量']
data_indices = [0, 1, 4, 9]
for ax, title, idx in zip(axs.flatten(), titles, data_indices):
    sns.barplot(x='word', y='prob', data=en_word_stats_list[idx].head(20), ax=ax)
    ax.set_title(title, fontsize=18)
    ax.set_xlabel('单词', fontsize=14)
    ax.set_ylabel('概率', fontsize=14)
    # 把坐标轴数值或者字母写大一点
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    
    # 显示数值
    for i, prob in enumerate(en_word_stats_list[idx]['prob'].head(20), 1):
        ax.text(i-1, prob, f'{prob:.4f}', fontsize=7, ha='center', va='bottom')


plt.tight_layout()
plt.savefig('en_word_distribution_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

In [17]:
# 利用单词验证zipf定律
# 通过绘制log-log图散点图以及进行线性拟合来验证zipf定律

rank = np.arange(1, len(en_word_stats_total) + 1)
word_prob = en_word_stats_total['prob'].values
log_rank = np.log(rank)
log_prob = np.log(word_prob)

# 绘制散点图
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

sns.scatterplot(x=log_rank, y=log_prob, edgecolor=None)

plt.title('英文单词Zipf定律验证散点图(全文本量)', fontsize=16)
plt.xlabel('log(排名)', fontsize=14)
plt.ylabel('log(概率)', fontsize=14)

plt.tight_layout()
plt.savefig('en_word_zipf_scatter.png', dpi=300, bbox_inches='tight')
plt.close()

In [18]:
X = sm.add_constant(log_rank)
y = log_prob
model = sm.OLS(y,X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                 7.905e+06
Date:                Thu, 30 Oct 2025   Prob (F-statistic):               0.00
Time:                        16:06:37   Log-Likelihood:                 6484.9
No. Observations:              240096   AIC:                        -1.297e+04
Df Residuals:                  240094   BIC:                        -1.295e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3638      0.005     66.190      0.000       0.353       0.375
x1            -1.3517      0.000  -2811.571      0.000      -1.353      -1.351
==============================================================================
Omnibus:                    15551.559   Durbin-Watson:                   0.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22011.596
Skew:                          -0.567   Prob(JB):                         0.00
Kurtosis:                       3.957   Cond. No.                         132.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
# 对 en_word_stats_list 的每个DataFrame做Zipf(log-log)线性拟合并汇总结果

results = []

for idx, df in enumerate(en_word_stats_list, start=1):
    label = f"{idx*10}%"  # 10%,20%...
    if df is None or df.empty:
        results.append({
            "segment": label, "n_words": 0,
            "slope": np.nan, "intercept": np.nan,
            "slope_se": np.nan, "slope_pvalue": np.nan,
            "r_squared": np.nan
        })
        continue
    
    df_sorted = df.sort_values(by='prob', ascending=False).reset_index(drop=True)

    n = len(df_sorted)

    # 计算排名和对数（natural log）
    ranks = np.arange(1, n + 1)
    probs = df_sorted['prob'].values
    log_rank = np.log(ranks)
    log_prob = np.log(probs)
    X = sm.add_constant(log_rank)
    model = sm.OLS(log_prob, X)
    res = model.fit()

    slope = res.params[1]
    intercept = res.params[0]
    slope_se = res.bse[1] if len(res.bse) > 1 else np.nan
    slope_p = res.pvalues[1] if len(res.pvalues) > 1 else np.nan
    r2 = res.rsquared

    results.append({
        "segment": label,
        "n_words": n,
        "slope": slope,
        "intercept": intercept,
        "slope_se": slope_se,
        "slope_pvalue": slope_p,
        "r_squared": r2
    })

results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv('zipf_fit_results_by_segment.csv', index=False)


  segment  n_words     slope  intercept  slope_se  slope_pvalue  r_squared
0     10%    55146 -1.228702  -0.553216  0.000904           0.0   0.971038
1     20%    85418 -1.278937  -0.169661  0.000742           0.0   0.972064
2     30%   109715 -1.306087   0.040378  0.000666           0.0   0.972252
3     40%   131832 -1.322507   0.172800  0.000616           0.0   0.972154
4     50%   152693 -1.330072   0.220296  0.000581           0.0   0.971679
5     60%   171788 -1.336260   0.260161  0.000552           0.0   0.971543
6     70%   190066 -1.341306   0.295594  0.000528           0.0   0.971361
7     80%   207991 -1.343995   0.308474  0.000510           0.0   0.970917
8     90%   223866 -1.349238   0.349420  0.000495           0.0   0.970800
9    100%   240096 -1.351668   0.363795  0.000481           0.0   0.970523


In [24]:
# 绘制斜率随文本量增加的变化图
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot(np.arange(1, 11) * 10, results_df['slope'], marker='o')
plt.title('拟合斜率随文本量变化图', fontsize=16)
plt.xlabel('文本量（%）', fontsize=14)
plt.ylabel('斜率', fontsize=14)

# 显示数值
for i, slope in enumerate(results_df['slope'], 1):
    plt.text(i * 10, slope, f'{slope:.4f}', fontsize=10, ha='center', va='bottom')
    
plt.tight_layout()
plt.savefig('en_word_zipf_slope_vs_text_size.png', dpi=300, bbox_inches='tight')
plt.close()

In [26]:
# 绘制R平方随文本量增加的变化图
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(8, 5))
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot(np.arange(1, 11) * 10, results_df['r_squared'], marker='o')
plt.title('拟合R平方随文本量变化图', fontsize=16)
plt.xlabel('文本量（%）', fontsize=14)
plt.ylabel('R方', fontsize=14)
# 显示数值
for i, r2 in enumerate(results_df['r_squared'], 1):
    plt.text(i * 10, r2, f'{r2:.4f}', fontsize=10, ha='center', va='bottom')

plt.tight_layout()
plt.savefig('en_word_zipf_r2_vs_text_size.png', dpi=300, bbox_inches='tight')
plt.close() 

In [30]:
# 绘制10%、20%、50%和100%文本量时候的四张英文单词Zipf定律验证散点图，用矩阵图展示
plt.style.use('default')

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

fig, axs = plt.subplots(2, 2, figsize=(20, 13))
# 设置子图标题
titles = ['10%文本量', '20%文本量', '50%文本量', '全文本量']
data_indices = [0, 1, 4, 9]
for ax, title, idx in zip(axs.flatten(), titles, data_indices):
    rank = np.arange(1, len(en_word_stats_list[idx]) + 1)
    word_prob = en_word_stats_list[idx]['prob'].values
    log_rank = np.log(rank)
    log_prob = np.log(word_prob)
    sns.scatterplot(x=log_rank, y=log_prob, edgecolor=None, ax=ax)
    ax.set_title(title, fontsize=18)
    ax.set_xlabel('log(排名)', fontsize=14)
    ax.set_ylabel('log(概率)', fontsize=14)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)

plt.tight_layout()
plt.savefig('en_word_zipf_scatter_comparison.png', dpi=300, bbox_inches='tight')
plt.close()